In [653]:
from collections import defaultdict
import copy

In [654]:
class BinaryTreeNode:

    def __init__(self, key, value, parent=None):
        self.key = key
        self.value = value
        self.lc = None
        self.rc = None
        self.parent = parent
        self.depth = 0
    
    def __repr__(self):
        return f"Key: {self.key}, Value: {self.value} "
    def setParent(self, parent):
        self.parent = parent

    def setDepth(self, depth):
        self.depth = depth

    def setLC(self, node):
        self.lc = node
        node.setParent(self)
    
    def setRC(self, node):
        self.rc = node
        node.setParent(self)
    
    def getValue(self):
        return self.value
    
    def getKey(self):
        return self.key
    
    def getDepth(self):
        return self.depth
    
    def hasParent(self):
        #True if self.parent != None else False
        if self.parent == None:
            return False
        return True
    
    def hasChildren(self):
        if self.rc == None and self.lc == None:
            return False
        return True
    
    def replaceChild(self, oldChild, newChild = None):
        if(oldChild.getKey() < self.getKey()):
            self.lc = newChild
        else:
            self.rc = newChild
    
        


In [655]:
class BSTRee:
    def __init__(self, key, value):
        self.root = BinaryTreeNode(key, value)
        self.allNodes = {key: self.root}
    
    def add(self, key, value, rootNode=None, depth=0, newNode = None):

        if newNode == None and self.hasKey(key):
            self.allNodes[key].value = value
            return
        
        if newNode == None:
            newNode = BinaryTreeNode(key, value)
            self.allNodes[key] = newNode
        
        
        if rootNode == None:
            rootNode = self.root
        
        depth += 1
        if newNode.getKey() < rootNode.getKey():
            self.addLeft(rootNode, newNode, depth)
        else:
            self.addRight(rootNode, newNode, depth)


    def addLeft(self, rootNode, newNode, depth):
        
        if rootNode.lc == None:
            newNode.depth = depth
            newNode.setParent(rootNode)
            rootNode.setLC(newNode)
            return
        
        self.add( None, None, rootNode=rootNode.lc, depth=depth, newNode=newNode)
    
    def addRight(self, rootNode, newNode, depth):
        
        if rootNode.rc == None:
            newNode.depth = depth
            newNode.setParent(rootNode)
            rootNode.setRC(newNode)

            return
        
        self.add(None, None,rootNode=rootNode.rc, depth=depth, newNode=newNode)

    def isRoot(self, node):
        return self.root.key == node.key
    
    
    def resetNode(self, oldNode, newNode):
        """
        if oldNode is root, replace root with new node
        otherwise, change old node's child from old node to new node
        """
        if self.isRoot(oldNode):
            self.root = newNode
            newNode.setParent(None)
            return
        oldNode.parent.replaceChild(oldNode, newChild = newNode)
        newNode.setParent(oldNode.parent)
        
    
    

    def removeNode(self, key):
        if not self.hasKey(key):
            raise Exception(f"Key not found: {key}")
        
        if self.size() == 1:
            self.root = None
            return self.allNodes.pop(key)

        removedNode = self.allNodes[key]
        isRoot = self.root.key == key

        #case 1: if node has no children
        if not removedNode.hasChildren():
            removedNode.parent.replaceChild(removedNode)
            return self.allNodes.pop(key)
        
        #case 3: if node has 2 children
        if removedNode.lc != None and removedNode.rc != None:
            successor = self.getSuccessor(removedNode)
            
            self.resetNode(removedNode, successor)
            return self.allNodes.pop(key)
        
        #case 2: if node has 1 child
        #subcase 1: 1 left child
        if removedNode.lc != None:
            self.resetNode(removedNode, removedNode.lc)
            removedNode.lc = None
            return self.allNodes.pop(key)
        
        #subcase 2: 1 right child
        if removedNode.rc != None:
            self.resetNode(removedNode, removedNode.rc)
            removedNode.rc = None
            return self.allNodes.pop(key)
        
    #fuck it, I'm too lazy to think of something more efficient
    def resetDepth(self, node):
        depth = 0
        curNode = node
        while curNode.hasParent():
            curNode = curNode.parent
            depth+=1
        node.setDepth(depth)
        return depth

    def resetAllDepth(self):
        for node in self.allNodes.values():
            self.resetDepth(node)

    def remove(self, key):
        self.removeNode(key)
        self.resetAllDepth()


    
    def getSuccessor(self, targetNode):

        successor = targetNode.rc
        while successor.lc != None:
            successor = successor.lc
        
        successor =  self.removeNode(successor.key)
        successor.lc = targetNode.lc
        successor.rc = targetNode.rc
        successor.lc.parent = successor
        successor.rc.parent = successor
        return successor

    
    def getDepth(self):
        return max(list(map(lambda node: node.getDepth(), self.allNodes.values())))
    
    def size(self):
        return len(self.allNodes)

    def hasKey(self, key):
        return True if key in self.allNodes else False
    
    def get(self, key):
        if not self.hasKey(key):
            raise Exception(f"Key not found: {key}")
        return self.allKVPs[key].getValue()
    



In [656]:
myTree = BSTRee(17, 17)
numbers = [ 10, 30, 5, 12, 18, 51, 19, 20]
for idx, number in enumerate(numbers):
    myTree.add(number, number)
myTree.remove(17)
print(myTree.root.depth)

2


In [ ]:
from collections import deque

def print_bst_visual(tree):
    if not tree.root:
        print("The tree is empty.")
        return

    # Queue for level-order traversal
    queue = deque([(tree.root, 0)])  # (node, level)
    current_level = 0
    level_nodes = []

    while queue:
        node, level = queue.popleft()

        # If we move to a new level, print the previous level
        if level != current_level:
            print("Level", current_level, ":", " ".join(level_nodes))
            level_nodes = []
            current_level = level

        # Add the current node's key to the level's list
        level_nodes.append(str(node.key))

        # Add children to the queue
        if node.lc:
            queue.append((node.lc, level + 1))
        if node.rc:
            queue.append((node.rc, level + 1))

    # Print the last level
    if level_nodes:
        print("Level", current_level, ":", " ".join(level_nodes))

# Example usage
myTree = BSTRee(17, 17)
numbers = [10, 30, 5, 12, 18, 51, 19, 20]
for number in numbers:
    myTree.add(number, number)

print_bst_visual(myTree)

       18      
       / \       
   10     30  
   / \      / \   
 5  12 19 51
             \      
     20


Prediction: The BST associated with the sorted list will require more time to insert and retrieve values than the BST associated with the randomly ordered list. This is because in the BST sorted list, each new node will be greater than the node previously added, so each node will be the right child of its parent node. This will make the tree only as efficient as a linked list, where adding a new item and retrieving an item will have a linear time complexity... If I coded the tree normally. Because I store each tree node in a hashmap where the key is a node's key, retrieving any node will take constant time, regardless of how the tree is structured. Adding an item to a tree created from a randomly sorted list will take logmarithmic time complexity at best and linear time complexity at worst, and retrieving an item from that tree will still take constant time. 